In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install rdkit
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 65.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import optuna
import lightgbm as lgb
import duckdb
import xgboost as xgb
from catboost import CatBoostClassifier
import numpy as np

In [4]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

In [5]:
con = duckdb.connect()

# 각 단백질에 대해 binds=0,1 데이터를 각각 12000개씩 불러오기
df_brd4_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'BRD4'
    ORDER BY random()
    LIMIT 12000
""").df()

df_brd4_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'BRD4'
    ORDER BY random()
    LIMIT 12000
""").df()

df_hsa_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'HSA'
    ORDER BY random()
    LIMIT 12000
""").df()

df_hsa_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'HSA'
    ORDER BY random()
    LIMIT 12000
""").df()

df_seh_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'sEH'
    ORDER BY random()
    LIMIT 12000
""").df()

df_seh_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'sEH'
    ORDER BY random()
    LIMIT 12000
""").df()

# 데이터프레임 결합
df = pd.concat([df_brd4_0, df_brd4_1, df_hsa_0, df_hsa_1, df_seh_0, df_seh_1], axis=0).reset_index(drop=True)
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
# ECFP 생성 함수
def generate_ecfp(molecule, radius=3, bits=2048):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

In [7]:
# 분자의 물리화학적 특성을 추출하는 함수
def generate_physchem_features(molecule):
    if molecule is None:
        return [np.nan] * 7
    return [
        Descriptors.MolWt(molecule),                   # 분자량
        Descriptors.MolLogP(molecule),                 # 로그 P
        Descriptors.NumHDonors(molecule),              # 수소 결합 공여자 수
        Descriptors.NumHAcceptors(molecule),           # 수소 결합 수용체 수
        Descriptors.TPSA(molecule),                    # TPSA (극성 표면적)
        Descriptors.NumRotatableBonds(molecule),       # 회전 가능한 결합 수
        Descriptors.RingCount(molecule)                # 고리의 수
    ]

In [8]:
# 데이터 전처리
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)
df['physchem'] = df['molecule'].apply(generate_physchem_features)
df['ecfp'] = df['molecule'].apply(generate_ecfp)

In [9]:
# 물리화학적 특성을 정규화
physchem_features = np.array(df['physchem'].tolist())
scaler = StandardScaler()
physchem_features_scaled = scaler.fit_transform(physchem_features)

In [10]:
# ECFP와 정규화된 물리화학적 특성을 결합
df['features'] = [ecfp + physchem_scaled.tolist() for ecfp, physchem_scaled in zip(df['ecfp'], physchem_features_scaled)]

In [11]:
# 단백질 이름을 원-핫 인코딩
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

In [12]:
# 최종 입력 데이터 생성
X = [features + list(protein) for features, protein in zip(df['features'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

In [13]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# LightGBM 하이퍼파라미터 최적화
def objective_lgb(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    map_score = average_precision_score(y_test, y_pred_proba)
    return map_score

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=50)
best_params_lgb = study_lgb.best_params

[I 2024-06-02 12:50:33,071] A new study created in memory with name: no-name-0242bb24-a65a-4b28-a2ab-21e1c0592f2a


[LightGBM] [Info] Number of positive: 28821, number of negative: 28779
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.028036 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4881
[LightGBM] [Info] Number of data points in the train set: 57600, number of used features: 2040
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500365 -> initscore=0.001458
[LightGBM] [Info] Start training from score 0.001458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2024-06-02 12:51:03,397] Trial 0 finished with value: 0.9572007582252896 and parameters: {'n_estimators': 124, 'max_depth': 6, 'learning_rate': 0.0859222580689771, 'subsample': 0.6682925681651174, 'colsample_bytree': 0.9488506342429788, 'num_leaves': 200}. Best is trial 0 with value: 0.9572007582252896.


[LightGBM] [Info] Number of positive: 28821, number of negative: 28779
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.488959 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4881
[LightGBM] [Info] Number of data points in the train set: 57600, number of used features: 2040
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500365 -> initscore=0.001458
[LightGBM] [Info] Start training from score 0.001458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2024-06-02 12:51:50,271] Trial 1 finished with value: 0.9677296258634012 and parameters: {'n_estimators': 225, 'max_depth': 8, 'learning_rate': 0.07373430538341771, 'subsample': 0.6720872823357572, 'colsample_bytree': 0.7313661762245987, 'num_leaves': 223}. Best is trial 1 with value: 0.9677296258634012.


...

[I 2024-06-02 13:24:49,794] Trial 47 finished with value: 0.9673079284320739 and parameters: {'n_estimators': 294, 'max_depth': 9, 'learning_rate': 0.04183752257545602, 'subsample': 0.9606132398741215, 'colsample_bytree': 0.9367182647208648, 'num_leaves': 221}. Best is trial 44 with value: 0.9723988412971686.


[LightGBM] [Info] Number of positive: 28821, number of negative: 28779
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.550563 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4881
[LightGBM] [Info] Number of data points in the train set: 57600, number of used features: 2040
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500365 -> initscore=0.001458
[LightGBM] [Info] Start training from score 0.001458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2024-06-02 13:25:31,943] Trial 48 finished with value: 0.9343248061280454 and parameters: {'n_estimators': 254, 'max_depth': 5, 'learning_rate': 0.017335936156931717, 'subsample': 0.8885571777813396, 'colsample_bytree': 0.9678156464776111, 'num_leaves': 243}. Best is trial 44 with value: 0.9723988412971686.


[LightGBM] [Info] Number of positive: 28821, number of negative: 28779
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.662824 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4881
[LightGBM] [Info] Number of data points in the train set: 57600, number of used features: 2040
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500365 -> initscore=0.001458
[LightGBM] [Info] Start training from score 0.001458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2024-06-02 13:26:12,662] Trial 49 finished with value: 0.9687966393064786 and parameters: {'n_estimators': 177, 'max_depth': 9, 'learning_rate': 0.09327481204825568, 'subsample': 0.8535591746892414, 'colsample_bytree': 0.9136464498184963, 'num_leaves': 252}. Best is trial 44 with value: 0.9723988412971686.


In [15]:
# XGBoost 하이퍼파라미터 최적화
def objective_xgb(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    map_score = average_precision_score(y_test, y_pred_proba)
    return map_score

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)
best_params_xgb = study_xgb.best_params

[I 2024-06-02 13:26:13,179] A new study created in memory with name: no-name-641016ea-76ec-424b-a770-413059e7b270
[I 2024-06-02 13:30:22,897] Trial 0 finished with value: 0.9709088779394104 and parameters: {'n_estimators': 222, 'max_depth': 10, 'learning_rate': 0.2023389591601935, 'subsample': 0.6709054059133913, 'colsample_bytree': 0.7306175080935562}. Best is trial 0 with value: 0.9709088779394104.
[I 2024-06-02 13:33:43,870] Trial 1 finished with value: 0.960872985690399 and parameters: {'n_estimators': 147, 'max_depth': 8, 'learning_rate': 0.05429783359347274, 'subsample': 0.7878962711205596, 'colsample_bytree': 0.9025281914806954}. Best is trial 0 with value: 0.9709088779394104.
[I 2024-06-02 13:37:03,012] Trial 2 finished with value: 0.9594618774521935 and parameters: {'n_estimators': 140, 'max_depth': 8, 'learning_rate': 0.04774858865521196, 'subsample': 0.614962471065997, 'colsample_bytree': 0.9821638024854997}. Best is trial 0 with value: 0.9709088779394104.
[I 2024-06-02 13:3

In [16]:
# CatBoost 하이퍼파라미터 최적화
def objective_cat(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
    }

    model = CatBoostClassifier(**param, verbose=0)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    map_score = average_precision_score(y_test, y_pred_proba)
    return map_score

study_cat = optuna.create_study(direction='maximize')
study_cat.optimize(objective_cat, n_trials=50)
best_params_cat = study_cat.best_params

[I 2024-06-02 16:49:51,446] A new study created in memory with name: no-name-88a7ede1-4cde-4d54-b937-fb210d65b462
[I 2024-06-02 16:52:23,651] Trial 0 finished with value: 0.9683394018448709 and parameters: {'iterations': 262, 'depth': 7, 'learning_rate': 0.2170632940657459, 'l2_leaf_reg': 0.07105089587273596, 'border_count': 198}. Best is trial 0 with value: 0.9683394018448709.
[I 2024-06-02 16:54:03,366] Trial 1 finished with value: 0.9369968393090451 and parameters: {'iterations': 145, 'depth': 3, 'learning_rate': 0.15782463446468453, 'l2_leaf_reg': 0.00520947801026005, 'border_count': 240}. Best is trial 0 with value: 0.9683394018448709.
[I 2024-06-02 16:56:07,193] Trial 2 finished with value: 0.9514326458963398 and parameters: {'iterations': 82, 'depth': 8, 'learning_rate': 0.10567422319758611, 'l2_leaf_reg': 0.1302810665457657, 'border_count': 41}. Best is trial 0 with value: 0.9683394018448709.
[I 2024-06-02 16:58:39,226] Trial 3 finished with value: 0.9155197867848557 and parame

In [17]:
# 최적의 파라미터로 모델 학습
model_lgb = lgb.LGBMClassifier(**best_params_lgb)
model_lgb.fit(X_train, y_train)

model_xgb = xgb.XGBClassifier(**best_params_xgb)
model_xgb.fit(X_train, y_train)

model_cat = CatBoostClassifier(**best_params_cat, verbose=0)
model_cat.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 28821, number of negative: 28779
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.527230 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4881
[LightGBM] [Info] Number of data points in the train set: 57600, number of used features: 2040
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500365 -> initscore=0.001458
[LightGBM] [Info] Start training from score 0.001458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [18]:
# 앙상블 예측 함수
def ensemble_predict_proba(models, X):
    predictions = np.zeros(len(X))
    for model in models:
        predictions += model.predict_proba(X)[:, 1]
    return predictions / len(models)

# 테스트 데이터 예측 및 저장
test_file = '/kaggle/input/leash-BELKA/test.csv'  
output_file = '/kaggle/working/submission6.csv'  # 출력 파일 경로

In [19]:
import os

models = [model_lgb, model_xgb, model_cat]

for df_test in pd.read_csv(test_file, chunksize=100000):
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)
    df_test['physchem'] = df_test['molecule'].apply(generate_physchem_features)
    
    # 테스트 데이터의 물리화학적 특성 정규화
    physchem_features_test = np.array(df_test['physchem'].tolist())
    physchem_features_test_scaled = scaler.transform(physchem_features_test)
    
    df_test['features'] = [ecfp + physchem_scaled.tolist() for ecfp, physchem_scaled in zip(df_test['ecfp'], physchem_features_test_scaled)]
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))
    X_test = [features + list(protein) for features, protein in zip(df_test['features'].tolist(), protein_onehot.tolist())]
    
    probabilities = ensemble_predict_proba(models, X_test)
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))